# 準備

## Google Colab 用の処理

下記を実行します
- ドライブのマウント
- ノートブックファイルと同じフォルダへの移動 

Googleドライブのマイドライブ を基準に DNN_code/DNN_code_colab_day2 フォルダを置くことを仮定しています。必要に応じて，パスを変更してください．

In [ ]:
# Google Colab での実行かを調べる
import sys
import os
ENV_COLAB = True  if 'google.colab' in sys.modules else False 

# google drive のマウント
if ENV_COLAB:
  from google.colab import drive 
  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/DNN_code/DNN_code_colab_day2/notebook')

## sys.pathの設定

In [ ]:
import sys
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定

# TensolFlow

## base

### constant

In [ ]:
import tensorflow as tf
import numpy as np
tf.config.run_functions_eagerly(False)

# それぞれ定数を定義
a = tf.constant(1)
b = tf.constant(2, dtype=tf.float32, shape=[3,2])
c = tf.constant(np.arange(4), dtype=tf.float32, shape=[2,2])

print('a:', a)
print('b:', b)
print('c:', c)


### placeholder

placeholder は、tensorflow ver.1 系にて
「構築した計算グラフが、"非定数値" (ex. モデルの入出力など) をどう扱うか」の情報を与えるために
用いられていたが、tensorflow ver.2 系では、ユーザーが明示的に使用することはほぼなくなった。
(参考までに tensoflow ver.1 のコードを行かにコメントアウトして残しておく)


- 参考:
  - [tensorflow.keras.input](https://www.tensorflow.org/api_docs/python/tf/keras/Input) などでは、内部的に placeholder の仕組みが使用されているようである。

In [ ]:
# import tensorflow as tf
# import numpy as np
# tf.config.run_functions_eagerly(False)

# # プレースホルダーを定義
# x = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None,3])

# print('x:', x)

# sess = tf.compat.v1.Session()

# X = np.random.rand(2,3)
# print('X:', X)

# # プレースホルダにX[0]を入力
# # shapeを(3,)から(1,3)にするためreshape
# print('x:', sess.run(x, feed_dict={x:X[0].reshape(1,-1)}))
# # プレースホルダにX[1]を入力
# print('x:', sess.run(x, feed_dict={x:X[1].reshape(1,-1)}))

### variables

In [ ]:
# 定数を定義
a = tf.constant(10)
print('a:', a)
# 変数を定義
x = tf.Variable(1)
print('x:', x)

# x * a の計算グラフを予め定義
@tf.function
def calc_x_by_a(x,a):
    return x * a
 
print(x.numpy())

x = calc_x_by_a(x,a)

print(x.numpy())

x = calc_x_by_a(x,a)

print(x.numpy())



## 線形回帰
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### [try]
-  noiseの値を変更しよう
-  dの数値を変更しよう
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
tf.config.run_functions_eagerly(False)

iters_num = 300
plot_interval = 10

# データを生成
n = 100
x = np.random.rand(n).astype(np.float32)
d = 3 * x + 2

# ノイズを加える
noise = 0.3
d = d + noise * np.random.randn(n).astype(np.float32)

# 最適化の対象の変数を初期化
W = tf.Variable(tf.zeros([1]))
b = tf.Variable(tf.zeros([1]))
trainable_variables= [W,b]

@tf.function
def linear_regression(xt, W, b):
    return W * xt + b

@tf.function
def calc_loss(y, dt):
    return tf.reduce_mean(input_tensor=tf.square(y - dt))

# 誤差関数 平均2乗誤差
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)


# 作成したデータをトレーニングデータとして準備
x_train = x.reshape(-1,1)
d_train = d.reshape(-1,1)

@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        y = linear_regression(inputs, W, b)
        loss = calc_loss(y, labels)
    gradients = tape.gradient(loss, trainable_variables) # 勾配 (loss を trainable_variables の各変数で微分) を求める
    optimizer.apply_gradients(zip(gradients, trainable_variables)) # 勾配を用いて変数の更新
    return loss

# %% トレーニング
iters_num = 100
plot_interval = 10

for i in range(iters_num):
    loss_val = train_step(x, d)
    if (i+1) % plot_interval == 0:
        print('Generation: ' + str(i+1) + '. 誤差 = ' + str(loss_val.numpy()))

# 学習された係数
print("W:", W.numpy())
print("b:", b.numpy())


# 回帰直線とデータのプロット
fig = plt.figure()
subplot = fig.add_subplot(1, 1, 1)
plt.scatter(x, d)
linex = np.linspace(0, 1, 2).astype(np.float32)
liney = linear_regression(linex, W, b)
subplot.plot(linex,liney)
plt.show()

## 非線形回帰
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### [try]
-  noiseの値を変更しよう
-  dの数値を変更しよう
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
tf.config.run_functions_eagerly(False)

# データを生成
n=100
x = np.random.rand(n).astype(np.float32) * 4 - 2
d =  - 0.4 * x ** 3 + 1.6 * x ** 2 - 2.8 * x + 1

#  ノイズを加える
noise = 0.05
d = d + noise * np.random.randn(n)

# %% 
# モデル
#  y = w0 + w1 * x + w2 * x^2 + w3 * x^3 
#  
#  note:
#  - 重みW については線形のため、モデル入力をベクトル [1, x, x^2, x^3] であるとする
#  - 線形回帰のバイアスは、w0 として W に含まれる

# numpy での予測関数
def predict(x,W_val):
    result = 0.
    for i in range(0,4):
        result += W_val[i,0] * x ** i
    return result

# 学習のための tensorflow のモデル関数
@tf.function
def model_function(xt, W):
    y=tf.matmul(xt,W)
    return y

# 誤差関数 平均２乗誤差
@tf.function 
def loss_fn(y,dt):
    loss = tf.reduce_mean(input_tensor=tf.square(y - dt))
    return loss

# 最適化の対象の変数を初期化
W = tf.Variable(tf.random.normal([4, 1], stddev=0.01))
trainable_variables = [W]

# %%  トレーニングループの定義
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        y = model_function(inputs, W)
        loss = loss_fn(y, labels)
    
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss

# %% 
optimizer = tf.optimizers.Adam(learning_rate=0.001)

# %%  トレーニングの実行
d_train = d.reshape(-1,1).astype(np.float32)
x_train = np.zeros([n, 4]).astype(np.float32)
for i in range(n):
    for j in range(4):
        x_train[i, j] = x[i]**j

#  トレーニング
iters_num = 10000
plot_interval = 100

for i in range(iters_num):
    loss_val = train_step(x_train, d_train)
    if (i+1) % plot_interval == 0:
        print('Generation: ' + str(i+1) + '. 誤差 = ' + str(loss_val.numpy()))

# 学習された係数
W_val = W.numpy() 
print("W:", W_val)

# 回帰直線とデータのプロット
fig = plt.figure()
subplot = fig.add_subplot(1, 1, 1)
plt.scatter(x, d)
linex = np.linspace(-2, 2, 100).astype(np.float32)
liney = predict(linex, W_val)
subplot.plot(linex,liney)
plt.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
## [try]
-  次の式をモデルとして回帰を行おう
$$ y=30x^{2} +0.5x+0.2 $$<br>
-  誤差が収束するようiters_numやlearning_rateを調整しよう
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
tf.config.run_functions_eagerly(False)

iters_num = 30000
plot_interval = 1000

input_layer_size = 3
output_layer_size = 1

# データを生成
n=100
x = np.random.rand(n).astype(np.float32) * 4 - 2
d = 30. * x ** 2 + 0.5 * x + 0.2

#  ノイズを加える
# noise = 0.05
# d = d + noise * np.random.randn(n)

# %% 
# モデル
#  y = \sigma_{i=0}^{N} ( w_i * x^i )  
#  
#  note:
#  - N = input_layer_size
#  - 重みW については線形のため、モデル入力をベクトル [1, x, x^2, x^3] であるとする
#  - 線形回帰のバイアスは、w0 として W に含まれる

# 予測関数
def predict(x):
    result = 0.
    for i in range(0,input_layer_size):
        result += W_val[i,0] * x ** i
    return result

# 学習のための tensorflow のモデル関数
@tf.function
def model_function(xt, W):
    y=tf.matmul(xt,W)
    return y

# 誤差関数 平均２乗誤差
@tf.function 
def loss_fn(y,dt):
    loss = tf.reduce_mean(input_tensor=tf.square(y - dt))
    return loss

# 最適化の対象の変数を初期化
W = tf.Variable(tf.random.normal([input_layer_size, 1], stddev=0.01))
trainable_variables = [W]

# %%  トレーニングループの定義
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        y = model_function(inputs, W)
        loss = loss_fn(y, labels)
    
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss


# 作成したデータをトレーニングデータとして準備
d_train = d.reshape([n, output_layer_size]).astype(np.float32)
x_train = np.zeros([n, input_layer_size]).astype(np.float32)
for i in range(n):
    for j in range(input_layer_size):
        x_train[i, j] = x[i]**j


#  トレーニング
# 誤差関数 平均２乗誤差
optimizer = tf.optimizers.Adam(0.01)

for i in range(iters_num):
    loss_val = train_step(x_train, d_train)
    if (i+1) % plot_interval == 0:
        print('Generation: ' + str(i+1) + '. 誤差 = ' + str(loss_val.numpy()))


# 学習された係数
W_val = W.numpy() 
print("W:", W_val[::-1])

fig = plt.figure()
subplot = fig.add_subplot(1,1,1)
plt.scatter(x ,d)
linex = np.linspace(-2,2,100)
liney = predict(linex)
subplot.plot(linex,liney)
plt.show()

## 分類1層 (mnist)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
## [try]
-  x：入力値, d：教師データ, W：重み, b：バイアス をそれぞれ定義しよう
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from data.mnist import load_mnist
import logging

# ロギングレベルの変更
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# 訓練用・テスト用(検証用) データの準備
(x_train, d_train), (x_test, d_test) = load_mnist(flatten=True, one_hot_label=True)
x_train = x_train.astype(np.float32)
d_train = d_train.astype(np.float32)

x_test = x_test.astype(np.float32)
d_test = d_test.astype(np.float32)

# %% モデル定義
@tf.function
def model_function(x):
    y = tf.nn.softmax(tf.matmul(x, W) + b)
    return y

# 最適化すべきパラメータ
W = tf.Variable(tf.random.normal([784, 10], stddev=0.01))
b = tf.Variable(tf.zeros([10]))
trainable_variables = [W, b]


# %% その他 tensorflow を用いて計算する関数の定義
# 誤差関数 (交差エントロピーの平均値)
@tf.function
def loss_fn(y, d):
    cross_entropy = -tf.reduce_sum(input_tensor=d * tf.math.log(y), axis=[1])
    loss = tf.reduce_mean(input_tensor=cross_entropy)
    return loss

# 正解率
@tf.function
def accuracy(y,d):
    correct = tf.equal(tf.argmax(input=y, axis=1), tf.argmax(input=d, axis=1))
    acc = tf.reduce_mean(tf.cast(correct, tf.float32))
    return acc



# 最適化手法は SGDを用いる
optimizer = tf.optimizers.SGD(learning_rate=0.1)

# %%  トレーニングループの定義
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model_function(inputs), labels)
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss

# %% 学習ループ
iters_num = 100
batch_size = 100
plot_interval = 1

accuracies = []
iters_per_epoch = len(x_train) // batch_size  
for i in range(iters_num):
    
    # epoch の最初でデータをシャッフルし、ミニバッチ取り出しの カウンターをリセット
    if i % iters_per_epoch == 0:
        x_train, d_train = shuffle(x_train, d_train, random_state=0)
        batch_count = 0

    # ミニバッチ取り出し
    start_ind = batch_size * batch_count 
    end_ind   = start_ind + batch_size
    x_batch = tf.Variable(x_train[start_ind:end_ind])
    d_batch = tf.Variable(d_train[start_ind:end_ind])
    batch_count +=1

    # 係数更新
    loss = train_step(x_batch, d_batch)

    # 1 回更新する毎に検証データに対する精度を計算
    if (i+1) % plot_interval == 0:
        # validation
        accuracy_val = accuracy(model_function( tf.Variable(x_test)), tf.Variable(d_test) ).numpy()
        accuracies.append(accuracy_val)
        print('Generation: ' + str(i+1) + '. 正解率 = ' + str(accuracy_val))


# 学習中の正答率の推移を表示
lists = range(0, iters_num, plot_interval)
plt.plot(lists, accuracies)
plt.title("accuracy")
plt.ylim(0, 1.0)
plt.show()

## 分類3層 (mnist)


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
## [try]
-  隠れ層のサイズを変更してみよう
-  optimizerを変更しよう<br>

tf.optimizers.SGD<br>
`__init__(
    learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD', **kwargs
)`

※ SGDはモメンタムの機能も兼ねる

tf.optimizers.Adagrad<br>
`__init__(
    learning_rate=0.001, initial_accumulator_value=0.1, epsilon=1e-07,
    name='Adagrad', **kwargs
)`

tf.optimizers.RMSPropOptimizer<br>
`__init__(
    learning_rate=0.001, 
    rho=0.9, 
    momentum=0.0, 
    epsilon=1e-07, 
    centered=False,
    name='RMSprop', 
    **kwargs
)`

tf.train.AdamOptimizer<br>
`__init__(
    learning_rate=0.001, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
     **kwargs
)`

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from data.mnist import load_mnist
import logging

# ロギングレベルの変更
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# 訓練用・テスト用(検証用) データの準備
(x_train, d_train), (x_test, d_test) = load_mnist(flatten=True, one_hot_label=True)
x_train = x_train.astype(np.float32)
d_train = d_train.astype(np.float32)

x_test = x_test.astype(np.float32)
d_test = d_test.astype(np.float32)

# %% モデル定義
# 最適化すべきパラメータ
hidden_layer_size_1 = 600
hidden_layer_size_2 = 300
dropout_rate = 0.5

W1 = tf.Variable(tf.random.normal([784, hidden_layer_size_1], stddev=0.01))
W2 = tf.Variable(tf.random.normal([hidden_layer_size_1, hidden_layer_size_2], stddev=0.01))
W3 = tf.Variable(tf.random.normal([hidden_layer_size_2, 10], stddev=0.01))
b1 = tf.Variable(tf.zeros([hidden_layer_size_1]))
b2 = tf.Variable(tf.zeros([hidden_layer_size_2]))
b3 = tf.Variable(tf.zeros([10]))
trainable_variables = [W1, b1, W2, b2, W3, b3]

# 推論関数
@tf.function
def model_function(x, keep_prob):
    z1 = tf.sigmoid(tf.matmul(x, W1) + b1)
    z2 = tf.sigmoid(tf.matmul(z1, W2) + b2)
    drop = tf.nn.dropout(z2, rate=1 - (keep_prob))
    y = tf.nn.softmax(tf.matmul(drop, W3) + b3)
    return y

# %% その他 tensorflow を用いた計算の定義

# 誤差関数 (交差エントロピーの平均値)
@tf.function
def loss_fn(y, d):
    cross_entropy = -tf.reduce_sum(input_tensor=d * tf.math.log(y), axis=[1])
    loss = tf.reduce_mean(input_tensor=cross_entropy)
    return loss

# 正解率
@tf.function
def accuracy(y,d):
    correct = tf.equal(tf.argmax(input=y, axis=1), tf.argmax(input=d, axis=1))
    acc = tf.reduce_mean(tf.cast(correct, tf.float32))
    return acc

# 最適化手法
# optimizer = tf.optimizers.SGD(learning_rate=0.5) # 純粋なSGD
# optimizer = tf.optimizers.SGD(learning_rate=0.1,momentum=0.9) # モメンタム
# optimizer = tf.optimizers.AdaGrad(learning_rate=0.1)
# optimizer = tf.optimizers.RMSprop(learning_rate=0.1)
optimizer = tf.optimizers.Adam(learning_rate=1e-4)


# %%  トレーニングループの定義
@tf.function
def train_step(inputs, labels, keep_prob):
    with tf.GradientTape() as tape:
        loss = loss_fn(model_function(inputs, keep_prob), labels)
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss

# %% 学習ループ
iters_num = 3000
batch_size = 100
plot_interval = 100

accuracies = []
iters_per_epoch = len(x_train) // batch_size  
for i in range(iters_num):
    
    # epoch の最初でデータをシャッフルし、ミニバッチ取り出しの カウンターをリセット
    if i % iters_per_epoch == 0:
        x_train, d_train = shuffle(x_train, d_train, random_state=0)
        batch_count = 0

    # ミニバッチ取り出し
    start_ind = batch_size * batch_count 
    end_ind   = start_ind + batch_size
    x_batch = tf.Variable(x_train[start_ind:end_ind])
    d_batch = tf.Variable(d_train[start_ind:end_ind])
    batch_count +=1

    # 係数更新
    loss = train_step(x_batch, d_batch, 1-dropout_rate)

    # 1 回更新する毎に検証データに対する精度を計算
    if (i+1) % plot_interval == 0:
        # validation
        accuracy_val = accuracy(model_function( tf.Variable(x_test), 1.0 ), tf.Variable(d_test) ).numpy()
        accuracies.append(accuracy_val)
        print('Generation: ' + str(i+1) + '. 正解率 = ' + str(accuracy_val))


# 学習中の正答率の推移を表示
lists = range(0, iters_num, plot_interval)
plt.plot(lists, accuracies)
plt.title("accuracy")
plt.ylim(0, 1.0)
plt.show()


## 分類CNN (mnist)
conv - relu - pool - conv - relu - pool - <br>
affin - relu - dropout - affin - softmax<br>

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### [try]
-  ドロップアウト率を0に変更しよう
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from data.mnist import load_mnist
import logging

# ロギングレベルの変更
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# 訓練用・テスト用(検証用) データの準備
# 全結合層のモデルと異なりベクトル化しない
(x_train, d_train), (x_test, d_test) = load_mnist(flatten=False, one_hot_label=True)
x_train = x_train.astype(np.float32)
d_train = d_train.astype(np.float32)

x_test = x_test.astype(np.float32)
d_test = d_test.astype(np.float32)


# これいるかどうか検討
x_train = np.reshape(x_train, [-1,28,28,1])
x_test = np.reshape(x_test, [-1,28,28,1])

# %% モデル定義
# 最適化すべきパラメータ
dropout_rate = 0.5
# dropout_rate = 0

# 第一層(畳み込み) のweightsとbiasのvariable
W_conv1 = tf.Variable(tf.random.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))

# 第二層(畳み込み) 
W_conv2 = tf.Variable(tf.random.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))

# 第三層(全結合)
W_fc1 = tf.Variable(tf.random.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

# 第四層(全結合)
W_fc2 = tf.Variable(tf.random.truncated_normal([1024, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))

trainable_variables = [W_conv1, b_conv1, W_conv2, b_conv2, W_fc1, b_fc1, W_fc2, b_fc2]

# 推論関数
@tf.function
def model_function(x, keep_prob):
    # 第一層のconvolutionalとpool
    # strides[0] = strides[3] = 1固定
    h_conv1 = tf.nn.relu(tf.nn.conv2d(input=x, filters=W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
    # プーリングサイズ n*n にしたい場合 ksize=[1, n, n, 1]
    h_pool1 = tf.nn.max_pool2d(input=h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 第二層のconvolutionalとpool
    h_conv2 = tf.nn.relu(tf.nn.conv2d(input=h_pool1, filters=W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
    h_pool2 = tf.nn.max_pool2d(input=h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 第一層と第二層でreduceされてできた特徴に対してrelu
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    # Dropout
    h_fc1_drop = tf.nn.dropout(h_fc1, rate=1 - (keep_prob))

    # 出来上がったものに対してSoftmax
    y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    return y_conv


# %% その他 tensorflow を用いた計算の定義

# 誤差関数 (交差エントロピーの平均値)
@tf.function
def loss_fn(y, d):
    cross_entropy = -tf.reduce_sum(input_tensor=d * tf.math.log(y), axis=[1])
    loss = tf.reduce_mean(input_tensor=cross_entropy)
    return loss

# 正解率
@tf.function
def accuracy(y,d):
    correct = tf.equal(tf.argmax(input=y, axis=1), tf.argmax(input=d, axis=1))
    acc = tf.reduce_mean(tf.cast(correct, tf.float32))
    return acc

# 最適化手法
# optimizer = tf.optimizers.SGD(learning_rate=0.5) # 純粋なSGD
# optimizer = tf.optimizers.SGD(learning_rate=0.1,momentum=0.9) # モメンタム
# optimizer = tf.optimizers.AdaGrad(learning_rate=0.1)
# optimizer = tf.optimizers.RMSprop(learning_rate=0.1)
optimizer = tf.optimizers.Adam(learning_rate=1e-4)


# %%  トレーニングループの定義
@tf.function
def train_step(inputs, labels, keep_prob):
    with tf.GradientTape() as tape:
        loss = loss_fn(model_function(inputs, keep_prob), labels)
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss

# %% 学習ループ
iters_num = 300
batch_size = 100
plot_interval = 10

accuracies = []
iters_per_epoch = len(x_train) // batch_size  
for i in range(iters_num):
    
    # epoch の最初でデータをシャッフルし、ミニバッチ取り出しの カウンターをリセット
    if i % iters_per_epoch == 0:
        x_train, d_train = shuffle(x_train, d_train, random_state=0)
        batch_count = 0

    # ミニバッチ取り出し
    start_ind = batch_size * batch_count 
    end_ind   = start_ind + batch_size
    x_batch = tf.Variable(x_train[start_ind:end_ind])
    d_batch = tf.Variable(d_train[start_ind:end_ind])
    batch_count +=1

    # 係数更新
    loss = train_step(x_batch, d_batch, 1-dropout_rate)

    # 1 回更新する毎に検証データに対する精度を計算
    if (i+1) % plot_interval == 0:
        # validation
        accuracy_val = accuracy(model_function( tf.Variable(x_test), 1.0 ), tf.Variable(d_test) ).numpy()
        accuracies.append(accuracy_val)
        print('Generation: ' + str(i+1) + '. 正解率 = ' + str(accuracy_val))


# 学習中の正答率の推移を表示
lists = range(0, iters_num, plot_interval)
plt.plot(lists, accuracies)
plt.title("accuracy")
plt.ylim(0, 1.0)
plt.show()